## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-11-01-10-40 +0000,startribune,Live: Walz expects ICE surge will soon wind do...,https://www.startribune.com/ice-raids-minnesot...
1,2026-02-11-01-09-04 +0000,nypost,Annie Guthrie’s street sees increased police p...,https://nypost.com/2026/02/10/us-news/annie-gu...
2,2026-02-11-01-08-45 +0000,nyt,‘No Reason He Should Have Died’: Alex Pretti’s...,https://www.nytimes.com/2026/02/10/us/alex-pre...
3,2026-02-11-01-00-00 +0000,wsj,Behind Trump’s Push to Remake the Kennedy Cent...,https://www.wsj.com/business/kennedy-center-do...
4,2026-02-11-01-00-00 +0000,wsj,Activist Investor Pushes Warner to Walk Away F...,https://www.wsj.com/business/deals/activist-in...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2438/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
32,trump,42
407,epstein,21
66,new,20
10,guthrie,13
3,ice,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
285,2026-02-10-11-32-07 +0000,nypost,Trump told Palm Beach police chief ‘thank good...,https://nypost.com/2026/02/10/us-news/trump-to...,107
354,2026-02-10-01-54-00 +0000,wsj,Trump Threatens to Block Opening of New Bridge...,https://www.wsj.com/world/americas/trump-threa...,101
129,2026-02-10-20-53-18 +0000,nyt,Former Palm Beach Police Chief Said Trump Told...,https://www.nytimes.com/2026/02/10/us/politics...,97
345,2026-02-10-02-41-35 +0000,nyt,Trump Threatens to Block Opening of New Bridge...,https://www.nytimes.com/2026/02/09/us/politics...,97
149,2026-02-10-20-05-05 +0000,bbc,Under-fire Trump commerce secretary confirms h...,https://www.bbc.com/news/articles/cwyx9vq0ne0o...,93


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
285,107,2026-02-10-11-32-07 +0000,nypost,Trump told Palm Beach police chief ‘thank good...,https://nypost.com/2026/02/10/us-news/trump-to...
167,75,2026-02-10-19-31-53 +0000,nyt,New Photos Show Masked Person on Nancy Guthrie...,https://www.nytimes.com/2026/02/10/us/nancy-gu...
233,48,2026-02-10-16-15-01 +0000,nypost,Gavin Newsom ripped by LA’s top fed prosecuto...,https://nypost.com/2026/02/10/us-news/gavin-ne...
20,39,2026-02-11-00-12-21 +0000,latimes,LAPD to train their body cameras on immigratio...,https://www.latimes.com/california/story/2026-...
191,39,2026-02-10-18-37-00 +0000,wsj,"Republican Sen. Susan Collins, the Maine centr...",https://www.wsj.com/politics/policy/susan-coll...
354,39,2026-02-10-01-54-00 +0000,wsj,Trump Threatens to Block Opening of New Bridge...,https://www.wsj.com/world/americas/trump-threa...
70,39,2026-02-10-22-47-54 +0000,nypost,Hochul hints NY won’t chip in more funding to ...,https://nypost.com/2026/02/10/us-news/hochul-w...
60,35,2026-02-10-23-07-36 +0000,nypost,"NYC muggers fatally pummel man, 66, – then try...",https://nypost.com/2026/02/10/us-news/nyc-mugg...
15,35,2026-02-11-00-25-19 +0000,startribune,Former federal prosecutor Joe Thompson joins D...,https://www.startribune.com/don-lemon-ice-minn...
124,34,2026-02-10-21-15-41 +0000,nypost,LI parents who tragically lost hockey star son...,https://nypost.com/2026/02/10/us-news/li-paren...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
